In [1]:
import tifffile as tiff
import numpy as np
from PIL import Image
from bs4 import BeautifulSoup

In [2]:
# Path to TIFF file
tif_file_path = './../data/Human_squamous_cell_carcinoma_stained_with_SignalStar_mIHC_technology/test/SCC_T37C_T1_Scan1_[10838,39481]_component_data.tif'

In [3]:
# Check if images can be opened as normal OME Tif files
with tiff.TiffFile(tif_file_path) as tif:
       img_ref=tif.pages[0].asarray()
       metadata=tif.ome_metadata
       print(metadata)

None


In [5]:
# OPTION 1 to extract tif files:
# IMPORTANT: You need to divide the positions to get the true tiles position. Be careful with the decimals.

# Read the TIFF file and extract metadata
with tiff.TiffFile(tif_file_path) as tif:
    for page_num, page in enumerate(tif.pages):
        print(f"\nMetadata for page {page_num + 1}:")

        # Iterate through all tags in the page
        for tag in page.tags.values():
            try:
                tag_name = tag.name
                tag_value = tag.value

                # Print the tag name and value
                print(f"{tag_name}: {tag_value}")

                # If the tag is 'ImageDescription', attempt to parse it with BeautifulSoup
                if tag_name == 'ImageDescription':
                    try:
                        ome = BeautifulSoup(tag_value, 'xml')
                        print("Parsed XML from ImageDescription:")
                        print(ome)
                        #print(ome.Image["ID"])
                    except Exception as e:
                        print(f"An error occurred while parsing ImageDescription metadata: {e}")

            except Exception as e:
                print(f"An error occurred while reading tag {tag.name}: {e}")



Metadata for page 1:
NewSubfileType: 0
ImageWidth: 1860
ImageLength: 1396
BitsPerSample: 32
Compression: 5
PhotometricInterpretation: 1
ImageDescription: <?xml version="1.0" encoding="utf-16"?>
<PerkinElmer-QPI-ImageDescription>
  <DescriptionVersion>3</DescriptionVersion>
  <AcquisitionSoftware>VectraPolaris</AcquisitionSoftware>
  <ImageType>FullResolution</ImageType>
  <Identifier>fb421a4c-d003-4eef-8a08-36854ee57c31</Identifier>
  <SlideID>SCC_T37C_T1_Scan1_[10838,39481]</SlideID>
  <Barcode />
  <ComputerName>DTC1D2VH3-7090</ComputerName>
  <IsUnmixedComponent>True</IsUnmixedComponent>
  <ExposureTime>9300</ExposureTime>
  <SignalUnits>33</SignalUnits>
  <Name>Alexa 488</Name>
  <Color>0,255,0</Color>
  <Responsivity>
    <Band>
      <Name>DAPI</Name>
      <Response>236.515433793925</Response>
      <Date>2023-05-23T13:39:10.0000000Z</Date>
      <FilterID>DAPI_Semrock:FF01-453/571/709-25 Emission / Semrock:FF01-391/538/649-25 Excitation</FilterID>
    </Band>
    <Band>
      

In [6]:
# Option 2 to get Position information

def get_tiff_metadata(tif_file_path):
    with Image.open(tif_file_path) as img:
        meta_dict = img.tag_v2
        for tag, value in meta_dict.items():
            print(f"Tag {tag}: {value}")


# Get metadata
get_tiff_metadata(tif_file_path)


Tag 256: 1860
Tag 257: 1396
Tag 258: (32,)
Tag 259: 5
Tag 262: 1
Tag 270: <?xml version="1.0" encoding="utf-16"?>
<PerkinElmer-QPI-ImageDescription>
  <DescriptionVersion>3</DescriptionVersion>
  <AcquisitionSoftware>VectraPolaris</AcquisitionSoftware>
  <ImageType>FullResolution</ImageType>
  <Identifier>fb421a4c-d003-4eef-8a08-36854ee57c31</Identifier>
  <SlideID>SCC_T37C_T1_Scan1_[10838,39481]</SlideID>
  <Barcode />
  <ComputerName>DTC1D2VH3-7090</ComputerName>
  <IsUnmixedComponent>True</IsUnmixedComponent>
  <ExposureTime>9300</ExposureTime>
  <SignalUnits>33</SignalUnits>
  <Name>Alexa 488</Name>
  <Color>0,255,0</Color>
  <Responsivity>
    <Band>
      <Name>DAPI</Name>
      <Response>236.515433793925</Response>
      <Date>2023-05-23T13:39:10.0000000Z</Date>
      <FilterID>DAPI_Semrock:FF01-453/571/709-25 Emission / Semrock:FF01-391/538/649-25 Excitation</FilterID>
    </Band>
    <Band>
      <Name>Opal 690</Name>
      <Response>0.198843662728369</Response>
      <Date>20

In [7]:
## Option 3 to extract Position information

# Open the TIFF file
with tiff.TiffFile(tif_file_path) as tif:
    # Iterate over all pages (IFDs) in the TIFF file
    for i, page in enumerate(tif.pages):
        print(f"=== TIFF Directory {i} ===")
        print(f"Image Width: {page.imagewidth}")
        print(f"Image Length: {page.imagelength}")
        print(f"Bits/Sample: {page.bitspersample}")

        # Extract resolution information
        x_resolution = page.tags.get('XResolution')
        y_resolution = page.tags.get('YResolution')
        if x_resolution and y_resolution:
            print(f"Resolution: {x_resolution.value}, {y_resolution.value}")

        # Extract position information (if available)
        x_position = page.tags.get('XPosition')
        y_position = page.tags.get('YPosition')
        if x_position and y_position:
            print(f"Position: {x_position.value}, {y_position.value}")

        # Extract and display metadata
        metadata = page.tags.get('ImageDescription')
        if metadata:
            print(f"ImageDescription: {metadata.value}")

        # Check for OME-XML metadata
        if page.is_ome:
            ome_metadata = page.ome_metadata
            print(f"OME-XML Metadata: {ome_metadata}")

        print()

# Display the OME-XML metadata of the first page if it exists
if tif.ome_metadata:
    print("=== OME-XML Metadata ===")
    print(tif.ome_metadata)


=== TIFF Directory 0 ===
Image Width: 1860
Image Length: 1396
Bits/Sample: 32
Resolution: (4294967295, 214253), (4294967295, 214253)
Position: (4294967295, 4140486705), (4294967295, 1097537948)
ImageDescription: <?xml version="1.0" encoding="utf-16"?>
<PerkinElmer-QPI-ImageDescription>
  <DescriptionVersion>3</DescriptionVersion>
  <AcquisitionSoftware>VectraPolaris</AcquisitionSoftware>
  <ImageType>FullResolution</ImageType>
  <Identifier>fb421a4c-d003-4eef-8a08-36854ee57c31</Identifier>
  <SlideID>SCC_T37C_T1_Scan1_[10838,39481]</SlideID>
  <Barcode />
  <ComputerName>DTC1D2VH3-7090</ComputerName>
  <IsUnmixedComponent>True</IsUnmixedComponent>
  <ExposureTime>9300</ExposureTime>
  <SignalUnits>33</SignalUnits>
  <Name>Alexa 488</Name>
  <Color>0,255,0</Color>
  <Responsivity>
    <Band>
      <Name>DAPI</Name>
      <Response>236.515433793925</Response>
      <Date>2023-05-23T13:39:10.0000000Z</Date>
      <FilterID>DAPI_Semrock:FF01-453/571/709-25 Emission / Semrock:FF01-391/538/6